# Aula 1 - Modelo de Regresión

## 1.1 Cargando los datos


[JSON Viewer](http://jsonviewer.stack.hu)

[Pandas: read_json](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html?highlight=read_json#pandas.read_json)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
datos = pd.read_json(
    path_or_buf='inmuebles.json',
    orient='columns'
)
datos

## 1.2 Explorando las informaciones

In [ ]:
datos.ident[0]

In [ ]:
datos.listing[0]

## 1.3 Tratamiento de datos iniciales

### Normalización

In [ ]:
datos_lista1 = pd.json_normalize(datos.ident)
datos_lista1.head()

In [ ]:
datos_lista2 = pd.json_normalize(datos.listing, sep='_')
datos_lista2.head()

### Juntando los dataframes

In [ ]:
datos_inmuebles = pd.concat([datos_lista1, datos_lista2], axis=1)
datos_inmuebles.head()

In [ ]:
datos_inmuebles.shape

In [ ]:
datos_inmuebles.to_json('inmueblesv1.json', orient='records', force_ascii=False)

In [ ]:
datos_inmuebles.address_zone.value_counts()

In [ ]:
for columna in datos_inmuebles.columns:
    print('========='*30)
    print(datos_inmuebles[columna].value_counts())

## 1.4 Limpieza

### Filtro

In [ ]:
filtro = (datos_inmuebles['types_usage'] == 'Residencial') & (datos_inmuebles['address_city'] ==  'Rio de Janeiro')

#Aplicando el filtro
datos_inmuebles = datos_inmuebles[filtro]
datos_inmuebles.head()

### Tipos de variables

In [ ]:
#Mucha atención al tipo de variables y al índice
datos_inmuebles.info()

Verificando la salida de `.info`

> 69588 entries, 0 to 73614

 Observamos que tenemos que resetear el index, para que las infos estén ordenadas correctamente.


In [ ]:
datos_inmuebles.reset_index(drop=True, inplace=True)

In [ ]:
#mostrar el RangeIndex
datos_inmuebles.info(verbose=False)

Ahora está correcto.

In [ ]:
datos_inmuebles = datos_inmuebles.astype({
    'prices_price': 'float64',
    'prices_tax_iptu': 'float64',
    'prices_tax_condo': 'float64',
    'features_usableAreas': 'int64',
    'features_totalAreas': 'int64',
})

datos_inmuebles.info()

## 1.5 - Tratamiento de datos finales

### Trabajando los nulos

#### Datos nulos zone

In [ ]:
datos_inmuebles.address_zone.value_counts()

In [ ]:
datos_inmuebles['address_zone'] = datos_inmuebles['address_zone'].replace('', np.nan)

In [ ]:
#verificación
datos_inmuebles.address_zone.isnull().sum()

In [ ]:
#relación zona con barrio
datos_inmuebles.head()

In [ ]:
dici = datos_inmuebles[~datos_inmuebles['address_zone'].isna()].drop_duplicates(subset=['address_neighborhood']).to_dict('records')
print(dici)

In [ ]:
#diccionario de barrio por zona
dic_zonas = {dic['address_neighborhood']: dic['address_zone'] for dic in dici}
print(dic_zonas)

In [ ]:
for barrio,zona in dic_zonas.items(): datos_inmuebles.loc[datos_inmuebles['address_neighborhood'] == barrio, 'address_zone'] = zona

In [ ]:
datos_inmuebles['address_zone'].isnull().sum()

#### Datos nulos prices

In [ ]:
datos_inmuebles.prices_tax_iptu.isnull().sum()

In [ ]:
datos_inmuebles.prices_tax_condo.isnull().sum()

In [ ]:
datos_inmuebles.prices_tax_iptu.fillna(0, inplace=True)
datos_inmuebles.prices_tax_condo.fillna(0, inplace=True)

In [ ]:
print(f"Total de prices nulos: {datos_inmuebles.prices_tax_iptu.isnull().sum()}")
print(f"Total de prices_tax_condo nulos: {datos_inmuebles.prices_tax_condo.isnull().sum()}")

### Modificar el nombre de las columnas y la selección de variables

In [ ]:
datos_inmuebles.drop(['customerID', 'source', 'types_usage', 'address_city',
                    'address_location_lon', 'address_location_lat','address_neighborhood' ], axis = 1, inplace=True)

In [ ]:
diccionario_columnas = {'types_unit': 'unit', 'address_zone': 'zone', 'prices_price': 'price',
                     'prices_tax_condo': 'tax_condo', 'prices_tax_iptu': 'tax_iptu', 'features_bedrooms': 'bedrooms',
                      'features_bathrooms': 'bathrooms', 'features_suites': 'suites', 'features_parkingSpaces': 'parkingSpaces',
                      'features_usableAreas': 'usableAreas', 'features_totalAreas':'totalAreas', 'features_floors':'floors',
                      'features_unitsOnTheFloor':'unitsOnTheFloor', 'features_unitFloor':'unitFloor'}

datos_inmuebles = datos_inmuebles.rename(diccionario_columnas, axis = 1)
datos_inmuebles.head()

# Aula 2 - Regresión Lineal

## 2.1 - Correlación

*   Correlación: grado de relación entre variables. Esta relación puede ser positiva o negativa.

*   Multicolinealidad: relación perfecta o exacta entre variables.

Para más detalles consultar [Documentación Seaborn](https://seaborn.pydata.org/generated/seaborn.heatmap.html)

Análisis de la distribución de la variable respuesta.

### Transformación de las variables

Estamos realizando una transformación logarítmica de las variables para obtener una distribución simétrica (más cercana a la normal), por lo que podemos usar el modelo de regresión lineal. La transformación realizada es:


$$log1p = log(1+x)$$


Recuerda realizar la transformación inversa para obtener el valor original:


$$expm1 = exp(x) - 1$$




* Para más detalles consultar las documentaciones [`log1p`](https://numpy.org/doc/stable/reference/generated/numpy.log1p.html), [`expm1`](https://numpy.org/doc/stable/reference/generated/numpy.expm1.html#numpy.expm1) e [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html#sklearn.preprocessing.FunctionTransformer)



### Dummies

## 2.2 - Regresión Lineal

* "El análisis de regresión se refiere al estudio de la dependencia de una variable, la variable dependiente, en relación con una o más variables, las variables explicativas, con el objetivo de estimar y/o predecir el valor medio (de la población) de la primera en términos de los valores conocidos o fijados (en muestreos repetidos) de estos últimos”.

* [Documentación Sklearn - LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

* Regresión simple: la variable dependiente (`y`) está relacionada con una única variable explicativa (`X`).

* Regresión múltiple: la variable dependiente (`y`) está relacionada con más de una única variable explicativa (`Xi`).

## 2.3 - Ajuste y Predicción

### División de los datos

### Modelo

## 2.4 - Métrica de desempeño

### Coeficiente de Determinación

* averiguar qué tan “bien” se ajusta una línea de regresión muestral a los datos
* r2 se encuentra entre 0 y 1
* "r2 mide la proporción o porcentaje de la variación total de Y explicada por el modelo de regresión".





### Extra

* [Documentación Sklearn - Métricas para Regresión](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

* Nota: ¡Estas métricas dependen de la escala de los datos!

* MSE (Mean Squared Error o Error cuadrático medio): media de la diferencia **elevada al cuadrado** entre el valor real y el predicho. *(penalización por error)*

* MAE (Mean Absolute Error o Error absoluto medio): media de la diferencia **absoluta** (módulo) entre el valor real y el predicho.


# Aula 3 - Arbol de Decisión para Regresión






## 3.1 - Implementando el Modelo


**Gráfico de error de predicción**: un gráfico de error de predicción muestra los objetivos reales frente a los valores predichos. En un modelo perfecto, estos puntos estarían alineados a 45 grados.


Para obtener más información sobre otras vistas:
[**Documentación de YellowBrick**](https://www.scikit-yb.org/en/latest/)

## 3.2 - Métrica

# Aula 4 - Random Forest para Regresión


## 4.1 - Modelo Ensemble

[Biblioteca Sklearn - Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)

## 4.2 - Predicción

**Observación**
Los parámetros son prácticamente los mismos entre `DecisionTreeRegressor()` y `RandomForestRegressor()`, con la excepción de:

* `n_estimators`: número de árboles en el bosque
* `bootstrap`: define si se utilizará el método de muestreo estadístico *bootstrap*
* `oob_score`: define si se utilizará el método de muestreo *out-of-bag* para medir los errores de predicción
* `n_jobs`: número de *jobs* que se ejecutan en paralelo (computación paralela - procesadores)
* `warm_start`: adecuación de un nuevo bosque o usa un bosque existente.
* `max_samples`: en caso de que `bootstrap` sea `True`, define el número de muestras que se extraerán de X para entrenar cada árbol base.

**Gráfico de error de predicción**

## 4.3 - Métrica de desempeño

# Aula 5 - Técnicas de Optimización


## 5.1 - Herramientas de Optimización

* ¿Por qué optimizar?

* ¿Qué herramientas tenemos?

* [Mejorando los hiperparámetros - Documentación Sklearn](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator)

### 1. Cross Validation (Validación Cruzada)

* [Documentación Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)

* **Cross Validation** es una técnica ampliamente utilizada para evaluar el rendimiento de los modelos y la consiguiente verificación de tendencias en el aprendizaje automático. Consiste en particionar los datos en subconjuntos (partes), donde un subconjunto se usa para entrenamiento y otro subconjunto se usa para probar y evaluar el rendimiento del modelo.

* ¡Los datos particionados son los datos de entrenamiento (70% de los datos originales)!

* **K-fold**: el conjunto de entrenamiento se divide aleatoriamente en K subconjuntos de aproximadamente el mismo tamaño, luego se realizan K iteraciones, donde los subconjuntos K-1 son para entrenamiento y 1 para validación.

* El rendimiento calculado por validación cruzada k-fold es el promedio de todos los valores calculados en cada iteración.

### 2. Randomized Parameter Optimization (Optimización Aleatoria de Parámetros)

* [Documentación Randomized Search](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-optimization)

* [Artículo científico sobre Random Search](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)

* "Define un espacio de búsqueda como un dominio acotado de valores de hiperparámetros y puntos de muestra aleatorios en ese dominio".

* Imagen en el artículo sobre cómo funciona esta búsqueda



## 5.2 - Optimizando el árbol de decisión

### Cross validation


### Random Search Validation

### Random Search Test

## 5.3 - Optimizando el Bosque de árboles de decisión

### Cross validation

### Random Search Validation

Definiendo los parámetros

### Random Search Test

## 5.4 - Escogiendo el mejor modelo